In [211]:
import matplotlib.pyplot as plt
import pylab
import math
import numpy as np
import scipy
import cmath
from scipy import special
from scipy import integrate
from numpy.polynomial.hermite import hermval
from numba import jit, njit, prange

In [267]:
fine_struct=0.0072973525693
num_morse_oscillators = 1
solvent_cutoff_freq = 0.0001
solvent_reorg = 0.0015530494095114032
is_emission = False
is_solvent = True
Ha_to_eV=27.211396132
kb_in_Ha=8.6173303*10.0**(-5.0)/Ha_to_eV
fs_to_Ha=2.418884326505*10.0**(-2.0)
max_t = 300
temp = 100
num_points = 4000
n = 20
expansion_number=127
gs_morse = 1 #number of gs Morse wavefunctions that will overlap with excited states, should be dependent on boltzmann distribution
ex_morse = 20

kbT=kb_in_Ha*temp
spectral_window=6
adiabatic = 2

n_max_gs = gs_morse
n_max_ex = ex_morse

#hartree energy unit
D_gs = 0.475217 #2.071824545e-18 J
D_ex = 0.20149  #8.78444853e-19 J

#bohr^-1 units
alpha_gs = 1.17199002 #6.20190409869e-11 meters^-1
alpha_ex = 1.23719524 #6.54695526318e-11 meters^-1

#bohr units'
shift_ex = 0.201444790427

#mass in a.u.
mu = 12178.1624678

'frequency in a.u.'
omega_gs = 0.01599479871665311 #6.61246943e14 Hz
omega_ex = 0.01099445915946515
# omega_gs = 0.010353662727838314 #4.28020304e14 rad (value when not divided by 2pi)


E_adiabatic = adiabatic/Ha_to_eV
E_spectral_window = spectral_window/Ha_to_eV

In [239]:
def spring_const(alpha,D):
	return (alpha**2)*2*D

def mode_freq(alpha_gs,D_gs,alpha_ex,D_ex,mu):
	omega_gs = np.sqrt(spring_const(alpha_gs,D_gs)/mu)/2*math.pi
	omega_ex = np.sqrt(spring_const(alpha_ex,D_ex)/mu)/2*math.pi
	return omega_gs,omega_ex

def compute_Harm_eval_n(omega,D,n):
	return omega*(n+0.5)-(omega**2/(4.0*D)*(n+0.5)**2.0)

def compute_morse_eval_n(omega,D,n):
	return omega*(n+0.5)-(omega*(n+0.5))**2.0/(4.0*D)

def find_classical_turning_points_morse(n_max_gs,n_max_ex,freq_gs,freq_ex,alpha_gs,alpha_ex,D_gs,D_ex,shift):
	E_max_gs=compute_morse_eval_n(freq_gs,D_gs,n_max_gs) # compute the energies for the highest energy morse 
	E_max_ex=compute_morse_eval_n(freq_ex,D_ex,n_max_ex) # state considered

	# find the two classical turning points for the ground state PES
	point1_gs=math.log(math.sqrt(E_max_gs/D_gs)+1.0)/(-alpha_gs)
	point2_gs=math.log(-math.sqrt(E_max_gs/D_gs)+1.0)/(-alpha_gs)

	# same for excited state. Include shift vector
	point1_ex=math.log(math.sqrt(E_max_ex/D_ex)+1.0)/(-alpha_ex)+shift
	point2_ex=math.log(-math.sqrt(E_max_ex/D_ex)+1.0)/(-alpha_ex)+shift

	# now find the smallest value and the largest value
	start_point=min(point1_gs,point2_gs)
	end_point=max(point1_ex,point2_ex)

	return start_point,end_point



In [229]:
omega_gs, omega_ex = mode_freq(alpha_gs,D_gs,alpha_ex,D_ex,mu)

print('start & end point:',find_classical_turning_points_morse(n_max_gs,n_max_ex,omega_gs,omega_ex,alpha_gs,alpha_ex,D_gs,D_ex,shift_ex))
start_point,end_point=find_classical_turning_points_morse(n_max_gs,n_max_ex,omega_gs,omega_ex,alpha_gs,alpha_ex,D_gs,D_ex,shift_ex)

#quantum turning points, established from 10% increase of classical tp's due to tunneling
start_point=start_point-start_point*0.1
end_point= end_point+end_point*0.1
step_x=(end_point-start_point)/num_points
x_range = np.arange(start_point,end_point,step_x)

start & end point: (-0.5729118537765941, 3.8103187625413644)


In [175]:
def Harm_wavefunc(x_range,omega,mu,n,shift):
	wavefunc=np.zeros((num_points,2))
	r_val=x_range*(math.sqrt(mu*omega))
	r_shift_val=shift*(math.sqrt(mu*omega))
	herm_coeffs = np.zeros(n+1)
	herm_coeffs[n] = 1
	norm=1/math.sqrt((2**n)*math.factorial(n))*(mu*omega/math.pi)**(0.25)
	#renormalize to prevent error buildup
	counter = 0.0
	for i in range(num_points):
		wavefunc[i,0]=start_point+counter*step_x
		counter = counter+1
	wavefunc[:,1] = norm*np.exp(-((r_val-r_shift_val)**2)/2)*np.polynomial.hermite.hermval(r_val-r_shift_val,herm_coeffs)
	psi_norm = integrate.simps(wavefunc[:,1]**2,dx=step_x)
	wavefunc[:,1] = wavefunc[:,1]/math.sqrt(psi_norm)
	
	return wavefunc

def Morse_wavefunc(num_points,start_point,end_point,D,alpha,mu,n,shift):
        # first start by filling array with position points:
        wavefunc=np.zeros((num_points,2))
        lamda=math.sqrt(2.0*D*mu)/(alpha)
        step_x=(end_point-start_point)/num_points
        denom=special.gamma(2.0*lamda-n)
        if np.isinf(denom):
                denom=10e280
        num=(math.factorial(n)*(2.0*lamda-2.0*n-1.0))
        normalization=math.sqrt(num/denom)
        counter=0
        for x in wavefunc:
                x[0]=start_point+counter*step_x
                r_val=(start_point+counter*step_x)*alpha
                r_shift_val=(shift)*alpha
                z_val=2.0*lamda*math.exp(-(r_val-r_shift_val))
                func_val=normalization*z_val**(lamda-n-0.5)*math.exp(-0.5*z_val)*special.assoc_laguerre(z_val,n,2.0*lamda-2.0*n-1.0)
                x[1]=func_val
                counter=counter+1

	# fix normalization regardless of value of denominator to avoid rounding errors
        wavefunc_sq=np.zeros(wavefunc.shape[0])
        wavefunc_sq[:]=wavefunc[:,1]*wavefunc[:,1]
        normalization=integrate.simps(wavefunc_sq,dx=step_x)
        for counter in range(wavefunc.shape[0]):
                wavefunc[counter,1]=wavefunc[counter,1]/math.sqrt(normalization)

        return wavefunc

**LC_coefficients function**

The LC_coefficients function below has two return outputs. The squared coefficcients are used as a failsafe to determine if your overlaps are caculated correctly. For any given function you are building with an LC of harmonic oscillators, the sum of their squared coefficients should approach 1 with increasing coeffs. For calculationsinvovling summing the coefficients you must use the "unsquared" list of coeffs.


In [266]:
def LC_coefficients(x_range,omega,num_points,start_point,end_point,D,alpha,mu,n,expansion_number,shift):
    morse = Morse_wavefunc(num_points,start_point,end_point,D,alpha,mu,n,shift)
    LC_coeffs = np.zeros(expansion_number)
    for i in range(expansion_number):
        LC_coeffs[i] = integrate.simps(morse[:,1]*Harm_wavefunc(x_range,omega,mu,i,shift)[:,1],x_range)
    return LC_coeffs


# coefficients = LC_coefficients(x_range,omega_ex,num_points,start_point,end_point,D_ex,alpha_ex,mu,n,expansion_number,shift_ex)
# plt.plot(coefficients)
print('coefficients:',LC_coefficients(x_range,omega_ex,num_points,start_point,end_point,D_ex,alpha_ex,mu,n,expansion_number,shift_ex))
print('coefficients sum SQ:',sum(LC_coefficients(x_range,omega_gs,num_points,start_point,end_point,D_ex,alpha_ex,mu,n,expansion_number,shift_ex)**2))
print('')

#sum of your squared overlap coefficients should approach convergence to 1 with increase in total states

coefficients: [ 8.36467544e-09 -1.51141656e-05 -2.31907762e-05  3.27444044e-04
  6.53356945e-04 -3.27351363e-03 -7.49689796e-03  1.89527854e-02
  4.67626540e-02 -7.16868496e-02 -1.64927219e-01  1.76692964e-01
  3.28438823e-01 -3.51976965e-01  2.63267080e-02 -1.92175259e-01
  3.50555075e-02  1.02539184e-01  1.28217500e-01  1.02302216e-01
  2.11861473e-02 -5.54490942e-02 -1.08174190e-01 -1.20883618e-01
 -9.44701574e-02 -4.24241089e-02  1.83243901e-02  7.12909479e-02
  1.04850673e-01  1.13775587e-01  9.88505416e-02  6.54503641e-02
  2.14863748e-02 -2.45864275e-02 -6.53337288e-02 -9.52832630e-02
 -1.11225036e-01 -1.12057322e-01 -9.84827829e-02 -7.28004824e-02
 -3.87447984e-02 -9.93376621e-04  3.58410904e-02  6.81965414e-02
  9.37734863e-02  1.11029015e-01  1.18792844e-01  1.16629256e-01
  1.05423694e-01  8.70851342e-02  6.35504189e-02  3.63877461e-02
  7.27777390e-03 -2.17534428e-02 -4.89681661e-02 -7.33158852e-02
 -9.38877056e-02 -1.09582880e-01 -1.19822787e-01 -1.24977293e-01
 -1.2548746

In [177]:
def Linear_combo_wfs(x_range,omega,D,alpha,mu,n,expansion_number,shift):
	coefficients=LC_coefficients(x_range,omega,num_points,start_point,end_point,D,alpha,mu,n,expansion_number,shift)
	LC_func = np.zeros((num_points,expansion_number))
	for i in range (expansion_number):
		LC_func[:,i]=Harm_wavefunc(x_range,omega,mu,i,shift)[:,1]*coefficients[i]
	#print(LC_func)
	return LC_func.sum(axis=1)

def LC_func_overlaps(x_range,omega_gs,omega_ex,D_ex,alpha_ex,D_gs,alpha_gs,mu,gs_morse,ex_morse,expansion_number,shift):
	gs_LC_morse=np.zeros((num_points,gs_morse))
	ex_LC_morse=np.zeros((num_points,ex_morse))
	gs_ex_Mat=np.zeros((gs_morse,ex_morse))
	counter=0
	step0=start_point
	step1=start_point+step_x
	for i in range(gs_morse):
		gs_LC_morse[:,i]=Linear_combo_wfs(x_range,omega_gs,D_gs,alpha_gs,mu,i,expansion_number,0.0) #start_point+counter*step_x
# 	print('gs_LC_Morse',gs_LC_morse)
	
	for j in range(ex_morse):
		ex_LC_morse[:,j] = Linear_combo_wfs(x_range,omega_ex,D_ex,alpha_ex,mu,j,expansion_number,shift)
# 	print('ex_LC_Morse',ex_LC_morse)
	
	for k in range(gs_morse):
		for l in range(ex_morse):
			overlap=gs_LC_morse[:,k]*ex_LC_morse[:,l]
			gs_ex_Mat[k,l]=integrate.simps(gs_LC_morse[:,k]*ex_LC_morse[:,l],dx=step1-step0)
	return gs_ex_Mat


In [227]:

def transition_energy(n_gs,n_ex):
	E_gs=compute_morse_eval_n(omega_gs,D_gs,n_gs)
	E_ex=compute_morse_eval_n(omega_ex,D_ex,n_ex)
	return E_ex-E_gs

def compute_morse_chi_func_t(omega_gs,D_gs,kbT,factors,energies,t):
        chi=0.0+0.0j
        Z=0.0 # partition function
        for n_gs in range(factors.shape[0]):
                Egs=compute_morse_eval_n(omega_gs,D_gs,n_gs)
                boltzmann=math.exp(-Egs/kbT)
                Z=Z+boltzmann
                for n_ex in range(factors.shape[1]):
                        chi=chi+boltzmann*factors[n_gs,n_ex]*cmath.exp(-1j*energies[n_gs,n_ex]*t)
	
        chi=chi/Z	

        return cmath.polar(chi)

**Sanity Check**
below the code verifies you are computing squared overlaps and transition energies.
Make sure the dimensions are correct and the math makes sense

In [ ]:
wf_overlaps=np.zeros((gs_morse,ex_morse))
wf_overlaps_sq=np.zeros((gs_morse,ex_morse))
boltzmann_fac=np.zeros((gs_morse))
transition_energies=np.zeros((gs_morse,ex_morse))
gs_energies=np.zeros(int(gs_morse))
ex_energies=np.zeros(int(ex_morse))

exact_response_func=np.zeros((1,1),dtype=np.complex_)
total_exact_response_func=np.zeros((1,1),dtype=np.complex_)
kbT=kb_in_Ha*temp

# wf_overlaps=LC_func_overlaps(x_range,omega_gs,omega_ex,D_ex,alpha_ex,D_gs,alpha_gs,mu,gs_morse,ex_morse,expansion_number,shift_ex)
# wf_overlaps_sq=wf_overlaps**2.0

def compute_exact_response_func(factors,energies,omega_gs,D_gs,kbT,max_t,num_points):
    step_length=max_t/num_points
    # end fc integral definition
    chi_full=np.zeros((num_points,3))
    response_func = np.zeros((num_points, 2), dtype=np.complex_)
    current_t=0.0
    for counter in range(num_points):
            chi_t=compute_morse_chi_func_t(omega_gs,D_gs,kbT,factors,energies,current_t)
            chi_full[counter,0]=current_t
            chi_full[counter,1]=chi_t[0]
            chi_full[counter,2]=chi_t[1]
            current_t=current_t+step_length

    # now make sure that phase is a continuous function:
    phase_fac=0.0
    for counter in range(num_points-1):
            chi_full[counter,2]=chi_full[counter,2]+phase_fac
            if abs(chi_full[counter,2]-phase_fac-chi_full[counter+1,2])>0.7*math.pi: #check for discontinuous jump.
                    diff=chi_full[counter+1,2]-(chi_full[counter,2]-phase_fac)
                    frac=diff/math.pi
                    n=int(round(frac))
                    phase_fac=phase_fac-math.pi*n
            chi_full[num_points-1,2]=chi_full[num_points-1,2]+phase_fac

    # now construct response function
    for counter in range(num_points):
            response_func[counter,0]=chi_full[counter,0]
            response_func[counter,1]=chi_full[counter,1]*cmath.exp(1j*chi_full[counter,2])

    return response_func

In [ ]:

# calculate transition energy between two specific morse oscillators.
def compute_exact_response(temp,omega_gs,omega_ex,max_t,num_steps):
	kbT=kb_in_Ha*temp
	for i in range(gs_morse):
		gs_energies[i]=compute_morse_eval_n(omega_gs,D_gs,i)
		for j in range(ex_morse):
			ex_energies[j]=compute_morse_eval_n(omega_ex,D_ex,j)+E_adiabatic
			transition_energies[i,j]=transition_energy(i,j)
	wf_overlaps=LC_func_overlaps(x_range,omega_gs,omega_ex,D_ex,alpha_ex,D_gs,alpha_gs,mu,gs_morse,ex_morse,expansion_number,shift_ex)
	wf_overlaps_sq=wf_overlaps**2.0
	exact_response_func=compute_exact_response_func(wf_overlaps_sq,transition_energies,omega_gs,D_gs,kbT,max_t,num_steps)
	
	return exact_response_func

In [182]:
print(compute_exact_response(temp,omega_gs,omega_ex,max_t,num_points))

KeyboardInterrupt: 

In [221]:
def solvent_spectral_dens(omega_cut,reorg,max_omega,num_steps):
	spectral_dens=np.zeros((num_steps,2))
	step_length=max_omega/num_steps
	counter=0
	omega=0.0
	while counter<num_steps:
		spectral_dens[counter,0]=omega
		# this definition of the spectral density guarantees that integrating the spectral dens yields the 
		# reorganization energy. We thus have a physical motivation for the chosen parameters
		spectral_dens[counter,1]=2.0*reorg*omega/((1.0+(omega/omega_cut)**2.0)*omega_cut)
		omega=omega+step_length
		counter=counter+1
	return spectral_dens

@jit(fastmath=True)
def integrant_2nd_order_cumulant_lineshape(spectral_dens,t_val,kbT):
	integrant=np.zeros((spectral_dens.shape[0],spectral_dens.shape[1]),dtype=np.complex_)
	for counter in range(spectral_dens.shape[0]):
		omega=spectral_dens[counter,0]
		integrant[counter,0]=omega
		if counter==0:
			integrant[counter,1]=0.0
		else:
			integrant[counter,1]=1.0/math.pi*spectral_dens[counter,1]/(omega**2.0)*(2.0*cmath.cosh(omega/(2.0*kbT))/cmath.sinh(omega/(2.0*kbT))*(math.sin(omega*t_val/2.0))**2.0+1j*(math.sin(omega*t_val)-omega*t_val))

	return integrant

# define the maximum number of t points this should be calculated for and the maximum number of steps
def compute_2nd_order_cumulant_from_spectral_dens(spectral_dens,kbT,max_t,steps):
    outfile = open('2nd_ord_cum_from_spec_den.dat','w')
    q_func=np.zeros((steps,2),dtype=complex)
    outfile.write('\n'+"Computing second order cumulant lineshape function."+'\n')
    outfile.write('\n'+'  Step       Time (fs)          Re[g_2]         Im[g_2]'+'\n')
    step_length=max_t/steps
    step_length_omega=spectral_dens[1,0]-spectral_dens[0,0]
    counter=0
    while counter<steps:
        t_current=counter*step_length
        q_func[counter,0]=t_current
        integrant=integrant_2nd_order_cumulant_lineshape(spectral_dens,t_current,kbT)
        q_func[counter,1]=integrate.simps(integrant[:,1],dx=(integrant[1,0]-integrant[0,0]))   #  give x and y axis
        outfile.write("%5d      %10.4f          %10.4e           %10.4e" % (counter,t_current*fs_to_Ha, np.real(q_func[counter,1]), np.imag(q_func[counter,1]))+'\n')
        counter=counter+1
    outfile.close()
    return q_func



spectral_dens=np.zeros((1,1))
g2_solvent=np.zeros((1,1))
solvent_response=np.zeros((1,1))

def calc_spectral_dens(num_points):
    spectral_dens=solvent_spectral_dens(solvent_cutoff_freq,solvent_reorg,solvent_cutoff_freq*20.0,num_points)
    return spectral_dens


def calc_g2_solvent(temp,num_points,max_t):
    spectral_dens = calc_spectral_dens(num_points)
    print('computing solvent lineshape function')
    kbT=kb_in_Ha*temp
    g2_solvent=compute_2nd_order_cumulant_from_spectral_dens(spectral_dens,kbT,max_t,num_points)
    return g2_solvent
    
def calc_solvent_response(is_emission):
    response_func = calc_spectral_dens(num_points)
    g2_solvent = calc_g2_solvent(temp,num_points,max_t)
    counter=0
    response_func=np.zeros((g2_solvent.shape[0],2),dtype=complex)
    while counter<g2_solvent.shape[0]:
        response_func[counter,0]=g2_solvent[counter,0].real
        if is_emission:
            response_func[counter,1]=cmath.exp(-np.conj(g2_solvent[counter,1]))
        else:
            response_func[counter,1]=cmath.exp(-g2_solvent[counter,1])
        counter=counter+1
    solvent_response=response_func
    print('SOLVENT RESPONSE VA', solvent_response)
    return solvent_response



In [ ]:
%%time
print(calc_spectral_dens(num_points))
print(calc_g2_solvent(temp,num_points,max_t))
print(calc_solvent_response(is_emission))

In [222]:
%%time
def spectrum_prefactor(Eval,is_emission):
	# prefactor alpha in Ha atomic units
	# Absorption: prefac=10*pi*omega*mu**2*alpha/(3*epsilon_0*ln(10))
	# Emission:   prefac=2*mu**2*omega**4*alpha**3/(3*epsilon_0)
	# note that 4pi*epslion0=1=> epslilon0=1/(4pi)

	prefac=0.0
	if not is_emission:
		# absorption constants
		prefac=40.0*math.pi**2.0*fine_struct*Eval/(3.0*math.log(10.0))
	else:
		# emission constants
		prefac=2.0*fine_struct**3.0*Eval**4.0*4.0*math.pi/3.0

	return prefac

def full_spectrum_integrant(response_func,solvent_response_func,E_val,is_solvent):
    integrant=np.zeros(response_func.shape[0])
    counter=0
    while counter<integrant.shape[0]:
        if is_solvent:
            integrant[counter]=(response_func[counter,1]*solvent_response_func[counter,1]*cmath.exp(1j*response_func[counter,0]*E_val)).real
        else:
            integrant[counter]=(response_func[counter,1]*cmath.exp(1j*response_func[counter,0]*E_val)).real
        counter=counter+1
    return integrant

def full_spectrum(response_func,solvent_response_func,steps_spectrum,start_val,end_val,is_solvent,is_emission):
	spectrum=np.zeros((steps_spectrum,2))
	counter=0
	spectrum_file = open('./Linear_combo_Full_Spec_vince.out', 'w')
	# print total response function
	spectrum_file.write('\n'+'Total Chromophore linear response function of the system:'+'\n')
	spectrum_file.write('\n'+'  Step       Time (fs)          Re[Chi]         Im[Chi]'+'\n')
	for i in range(response_func.shape[0]):
		spectrum_file.write("%5d      %10.4f          %10.4e       %10.4e" % (i+1,np.real(response_func[i,0])*fs_to_Ha, np.real(response_func[i,1]), np.imag(response_func[i,1]))+'\n')

	spectrum_file.write('\n'+'Computing linear spectrum of the system between '+str(start_val*Ha_to_eV)+' and '+str(end_val*Ha_to_eV)+' eV.')
	spectrum_file.write('\n'+'Total linear spectrum of the system:'+'\n')
	spectrum_file.write('Energy (Ha)         Absorbance (Ha)'+'\n')	
	step_length=((end_val-start_val)/steps_spectrum)
	while counter<spectrum.shape[0]:
		E_val=start_val+counter*step_length
		prefac=spectrum_prefactor(E_val,is_emission)
		integrant=full_spectrum_integrant(response_func,solvent_response_func,E_val,is_solvent)
		spectrum[counter,0]=E_val
		spectrum[counter,1]=prefac*(integrate.simps(integrant,dx=response_func[1,0].real-response_func[0,0].real))
		spectrum_file.write("%2.5f          %10.4e" % (spectrum[counter,0], spectrum[counter,1])+'\n') 
		counter=counter+1

	spectrum[:,0]=spectrum[:,0]*Ha_to_eV
	spectrum_file.close()
	return spectrum

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 22.4 µs


In [268]:
%%time

response_func = compute_total_exact_response(temp,max_t,num_points)
solvent_response_func = calc_solvent_response(is_emission)
spectrum=np.zeros((num_points,2))
E_start=adiabatic-spectral_window/2.0
E_end=adiabatic+spectral_window/2.0
counter = 0
step_length=((E_end-E_start)/num_points)




Computed response func!
[[0.00000000e+00+0.j         9.99999982e-01+0.j        ]
 [7.50000000e-02+0.j         9.99997785e-01-0.00166451j]
 [1.50000000e-01+0.j         9.99991194e-01-0.00332901j]
 ...
 [2.99775000e+02+0.j         1.01568023e-02+0.01573114j]
 [2.99850000e+02+0.j         1.01353207e-02+0.01575108j]
 [2.99925000e+02+0.j         1.01138172e-02+0.01577101j]]
computing solvent lineshape function
[[0.00000000e+00+0.00000000e+00j 1.00000000e+00-0.00000000e+00j]
 [7.50000000e-02+0.00000000e+00j 9.99999997e-01+1.28428962e-14j]
 [1.50000000e-01+0.00000000e+00j 9.99999988e-01+1.02743170e-13j]
 ...
 [2.99775000e+02+0.00000000e+00j 9.54754868e-01+7.77978630e-04j]
 [2.99850000e+02+0.00000000e+00j 9.54732781e-01+7.78542189e-04j]
 [2.99925000e+02+0.00000000e+00j 9.54710688e-01+7.79106005e-04j]]
CPU times: user 9.46 s, sys: 3.89 ms, total: 9.46 s
Wall time: 9.46 s


In [223]:
def compute_total_exact_response(temp,max_t,num_points):
	for i in range(num_morse_oscillators):
		exact_response_func = compute_exact_response(temp,omega_gs,omega_ex,max_t,num_points)
		print('Computed response func!')
		print(exact_response_func)
		total_exact_response_func=exact_response_func
	for j in range(total_exact_response_func.shape[0]):
		total_exact_response_func[j,1]=total_exact_response_func[j,1]*cmath.exp(-1j*E_adiabatic*total_exact_response_func[j,0])
	
	return total_exact_response_func

In [224]:

def compute_morse_absorption(is_emission,temp,num_points,max_t):
	# first compute solvent response. This is NOT optional for the Morse oscillator, same
	solvent_response = calc_solvent_response(is_emission)
	# figure out start and end values over which we compute the spectrum
	# at the moment this is a Hack because we have no expression to analytically 
	# evaluate the average energy gap of the Morse oscillator. 
	E_start=adiabatic-spectral_window/2.0
	E_end=adiabatic+spectral_window/2.0

	# exact solution to the morse oscillator
	total_exact_response_func = compute_total_exact_response(temp,max_t,num_points)
	print('total exact response func')
	print(total_exact_response_func.shape)
	print(total_exact_response_func)
	spectrum=full_spectrum(total_exact_response_func,solvent_response,num_points,E_start,E_end,True,False)
	np.savetxt('Morse_exact_spectrum.dat', spectrum, header='Energy (eV)      Intensity (arb. units)')


linear_absorption = compute_morse_absorption(is_emission,temp,num_points,max_t)

computing solvent lineshape function
[[0.00000000e+00+0.00000000e+00j 1.00000000e+00-0.00000000e+00j]
 [7.50000000e-02+0.00000000e+00j 9.99999997e-01+1.28428962e-14j]
 [1.50000000e-01+0.00000000e+00j 9.99999988e-01+1.02743170e-13j]
 ...
 [2.99775000e+02+0.00000000e+00j 9.54754868e-01+7.77978630e-04j]
 [2.99850000e+02+0.00000000e+00j 9.54732781e-01+7.78542189e-04j]
 [2.99925000e+02+0.00000000e+00j 9.54710688e-01+7.79106005e-04j]]
Computed response func!
[[0.00000000e+00+0.j         1.00000211e+00+0.j        ]
 [7.50000000e-02+0.j         9.99999846e-01-0.00169036j]
 [1.50000000e-01+0.j         9.99993051e-01-0.00338071j]
 ...
 [2.99775000e+02+0.j         8.67435256e-03+0.01703805j]
 [2.99850000e+02+0.j         8.65103228e-03+0.01705728j]
 [2.99925000e+02+0.j         8.62768898e-03+0.0170765j ]]
total exact response func
(4000, 2)
[[ 0.00000000e+00+0.j          1.00000211e+00+0.j        ]
 [ 7.50000000e-02+0.j          9.99975335e-01-0.00720271j]
 [ 1.50000000e-01+0.j          9.99895008